In [25]:
import pandas as pd
import math
import ast

pd.set_option('display.max_columns', None)

authors = pd.read_csv("./processed_authors.csv", delimiter=",")
authors = authors.drop(columns=['IDX', 'FULL_NAME', 'FIRST_LAST_NAME'])
authors


,ID,NM_AUTOR,NM_ABNT_AUTOR,TP_AUTOR,NM_TP_CATEGORIA_DOCENTE,NM_NIVEL_DISCENTE,CD_PROGRAMA_IES,NM_PROGRAMA_IES,NM_AREA_CONHECIMENTO,SG_ENTIDADE_ENSINO,ID_ADD_PRODUCAO_INTELECTUAL
0,1472066.0,{'SANDRA PATRICIA DE ANDRADE MARIANO SANTOS': 1},"{'SANTOS, S. P. A. M.': 1}",{'DISCENTE': 1},{'nan': 1},{'MESTRADO PROFISSIONAL': 1},{'33002010225P1': 1},{'TECNOLOGIA EM QUÍMICA E BIOQUÍMICA': 1},{'nan': 1},{'USP': 1},[21957132]
1,18519.0,{'KOITI ARAKI': 20},"{'ARAKI, K.': 20}","{'PARTICIPANTE EXTERNO': 19, 'DOCENTE': 1}","{'nan': 19, 'PERMANENTE': 1}",{'nan': 20},"{'33063010002P3': 12, '33063010005P2': 6, '330...","{'ODONTOLOGIA': 13, 'PATOLOGIA AMBIENTAL E EXP...",{'nan': 20},"{'UNIP': 18, 'USP': 2}","[21957136, 22229203, 22229231, 22229260, 22229..."
2,587155.0,{'MIKHAEL KIMITOSHI ABE': 1},"{'ABE, M. K.': 1}",{'DISCENTE': 1},{'nan': 1},{'MESTRADO PROFISSIONAL': 1},{'33002010225P1': 1},{'TECNOLOGIA EM QUÍMICA E BIOQUÍMICA': 1},{'nan': 1},{'USP': 1},[21957138]
3,2546681.0,{'TATIANA VALERIA FURQUIM': 1},"{'FURQUIM, T. V.': 1}",{'DISCENTE': 1},{'nan': 1},{'MESTRADO PROFISSIONAL': 1},{'33002010225P1': 1},{'TECNOLOGIA EM QUÍMICA E BIOQUÍMICA': 1},{'nan': 1},{'USP': 1},[21957139]
4,1094327.0,{'THIAGO LUIZ MORAES MONTEIRO': 1},"{'MONTEIRO, T. L. M.': 1}",{'DISCENTE': 1},{'nan': 1},{'MESTRADO PROFISSIONAL': 1},{'33002010225P1': 1},{'TECNOLOGIA EM QUÍMICA E BIOQUÍMICA': 1},{'nan': 1},{'USP': 1},[21957148]
...,...,...,...,...,...,...,...,...,...,...,...
645420,NaN,{'ROSIERE FONTELES DE ARAUJO': 1},"{'ARAUJO, ROSIERE FONTELES DE': 1}",{'-': 1},{'nan': 1},{'nan': 1},{'13003011002P2': 1},{'AGROECOLOGIA': 1},{'AGRONOMIA': 1},{'UERR': 1},[27089012]
645421,NaN,{'LUIZ ALBERTO OLIVEIRA': 1},"{'OLIVEIRA, L. A.': 1}",{'-': 1},{'nan': 1},{'nan': 1},{'31005012013P8': 1},{'ENGENHARIA MATERIAIS E DE PROCESSOS QUÍMICOS...,{'FÍSICA': 1},{'PUC-RIO': 1},[27087426]
645422,NaN,{'WASHINGTON ALLYSSON DANTAS SILVA': 1},"{'SILVA, W. A. D.': 1}",{'-': 1},{'nan': 1},{'nan': 1},{'24001015059P2': 1},{'GESTÃO EM ORGANIZAÇÕES APRENDENTES': 1},{'PSICOLOGIA': 1},{'UFPB-JP': 1},[27086287]
645423,NaN,{'JONHSON DE TARSO SILVA': 1},"{'SILVA, JONHSON DE TARSO': 1}",{'-': 1},{'nan': 1},{'nan': 1},{'33005010032P1': 1},{'TECNOLOGIAS DA INTELIGÊNCIA E DESIGN DIGITAL...,{'CIÊNCIA DA COMPUTAÇÃO': 1},{'PUC/SP': 1},[27086690]


In [26]:
dict_fields = [
  'NM_AUTOR',
  'NM_ABNT_AUTOR',
  'CD_PROGRAMA_IES',
  'NM_PROGRAMA_IES',
  'NM_AREA_CONHECIMENTO',
  'SG_ENTIDADE_ENSINO',
]

def parse_array(s):
  try:
    lit = ast.literal_eval(s)  
  except:
    return []

  return lit

def most_freq(d):
  try:
    d = ast.literal_eval(d)  
    x = -1
    k = None
    for key in d:
      if (d[key] > x and key != 'nan'):
        x = d[key]
        k = key
  except:
    return '-'

  return k if k != 'nan' else '-'

def select_by_priority(priority_list, d):
  try:
    d = ast.literal_eval(d)
    n = next((type for type in priority_list if type in d.keys()), None)
  except:
    # print(d)
    # print(type(d))
    n = None
  return n 
  
def priority(priority_list):
  return lambda d: select_by_priority(priority_list, d)
  
authors[['ID_ADD_PRODUCAO_INTELECTUAL']] = authors[['ID_ADD_PRODUCAO_INTELECTUAL']].applymap(parse_array)
authors[['TP_AUTOR']] = authors[['TP_AUTOR']].applymap(priority(['DOCENTE', 'EGRESSO', 'PÓS-DOC', 'DISCENTE', 'PARTICIPANTE EXTERNO']))
authors[['NM_TP_CATEGORIA_DOCENTE']] = authors[['NM_TP_CATEGORIA_DOCENTE']].applymap(priority(['PERMANENTE', 'COLABORADOR', 'VISITANTE']))
authors[['NM_NIVEL_DISCENTE']] = authors[['NM_NIVEL_DISCENTE']].applymap(priority(['DOUTORADO PROFISSIONAL', 'BACHARELADO', 'MESTRADO', 'DOUTORADO', 'MESTRADO PROFISSIONAL']))
authors[dict_fields] = authors[dict_fields].applymap(most_freq)

authors

,ID,NM_AUTOR,NM_ABNT_AUTOR,TP_AUTOR,NM_TP_CATEGORIA_DOCENTE,NM_NIVEL_DISCENTE,CD_PROGRAMA_IES,NM_PROGRAMA_IES,NM_AREA_CONHECIMENTO,SG_ENTIDADE_ENSINO,ID_ADD_PRODUCAO_INTELECTUAL
0,1472066.0,SANDRA PATRICIA DE ANDRADE MARIANO SANTOS,"SANTOS, S. P. A. M.",DISCENTE,None,MESTRADO PROFISSIONAL,33002010225P1,TECNOLOGIA EM QUÍMICA E BIOQUÍMICA,None,USP,[21957132]
1,18519.0,KOITI ARAKI,"ARAKI, K.",DOCENTE,PERMANENTE,None,33063010002P3,ODONTOLOGIA,None,UNIP,"[21957136, 22229203, 22229231, 22229260, 22229..."
2,587155.0,MIKHAEL KIMITOSHI ABE,"ABE, M. K.",DISCENTE,None,MESTRADO PROFISSIONAL,33002010225P1,TECNOLOGIA EM QUÍMICA E BIOQUÍMICA,None,USP,[21957138]
3,2546681.0,TATIANA VALERIA FURQUIM,"FURQUIM, T. V.",DISCENTE,None,MESTRADO PROFISSIONAL,33002010225P1,TECNOLOGIA EM QUÍMICA E BIOQUÍMICA,None,USP,[21957139]
4,1094327.0,THIAGO LUIZ MORAES MONTEIRO,"MONTEIRO, T. L. M.",DISCENTE,None,MESTRADO PROFISSIONAL,33002010225P1,TECNOLOGIA EM QUÍMICA E BIOQUÍMICA,None,USP,[21957148]
...,...,...,...,...,...,...,...,...,...,...,...
645420,NaN,ROSIERE FONTELES DE ARAUJO,"ARAUJO, ROSIERE FONTELES DE",None,None,None,13003011002P2,AGROECOLOGIA,AGRONOMIA,UERR,[27089012]
645421,NaN,LUIZ ALBERTO OLIVEIRA,"OLIVEIRA, L. A.",None,None,None,31005012013P8,ENGENHARIA MATERIAIS E DE PROCESSOS QUÍMICOS E...,FÍSICA,PUC-RIO,[27087426]
645422,NaN,WASHINGTON ALLYSSON DANTAS SILVA,"SILVA, W. A. D.",None,None,None,24001015059P2,GESTÃO EM ORGANIZAÇÕES APRENDENTES,PSICOLOGIA,UFPB-JP,[27086287]
645423,NaN,JONHSON DE TARSO SILVA,"SILVA, JONHSON DE TARSO",None,None,None,33005010032P1,TECNOLOGIAS DA INTELIGÊNCIA E DESIGN DIGITAL,CIÊNCIA DA COMPUTAÇÃO,PUC/SP,[27086690]


In [27]:
authors = authors.groupby(['NM_AUTOR'], sort=False, as_index=False).agg({
  'ID': 'first',
  'NM_AUTOR': 'first',
  'NM_ABNT_AUTOR': 'first',
  'TP_AUTOR': 'first',
  'NM_TP_CATEGORIA_DOCENTE': 'first',
  'NM_NIVEL_DISCENTE': 'first',
  'CD_PROGRAMA_IES': 'first',
  'NM_PROGRAMA_IES': 'first',
  'NM_AREA_CONHECIMENTO': 'first',
  'SG_ENTIDADE_ENSINO': 'first',
  'ID_ADD_PRODUCAO_INTELECTUAL': 'sum'
})

In [28]:
authors.to_csv('final_authors.csv', sep=';')
authors.to_excel('final_authors.xlsx')